# Phys 581 Winter 2019
# Assignment #2: Geomagnetic Bottle
## Alexander Hickey, 10169582

In [1]:
#Import useful libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

##### Task: 
Read  https://www2.mps.mpg.de/solar-system-school/lectures/space_plasma_physics_2007/Lecture_3.pdf and then numerically integrate the motion of a 1keV proton starting at the surface of the Earth ($r=6371km$) with latitude $66^\circ$ moving antiparallel to the local $\vec B$.  Assume the geomagnetic field is a dipole aligned with the spin axis with magnetic moment $m=8.05E22 A m^2$.

Does you result look like the plot in slide#7?  If not, what values (eg. energy, pitch angle, latitude) produce a better resemblance?

Magnetic field of a dipole in standard spherical coordinates:
$$\mathbf B(\mathbf r) = \frac{\mu_0}{4\pi} \frac{M_E}{|\mathbf r|^3} \left( 2\cos{\theta} \hat r  + \sin \theta \mathbf{\hat{\theta}} \right)$$

where $M_E = 8.05 \times 10^{22} \text{ Am}^2$ is the magnetic dipole moment of the earth, and $\mu_0= 4\pi \times 10^7 \text{ Tm/A}$ is the permeability of free space.
Working with these numbers numbers directly will most likely lead to floating point error, so we start by making the equations of motion dimensionless. We will measure all distances in units of the radius of earth $R_E = 6371 \text{ km}$. Define a base unit of magnetic flux density

$$B_0 := \frac{\mu_0}{4 \pi} \frac{M_E}{R_E^3}$$

then we can rewrite the Earth's magnetic field as 

$$\frac{\mathbf B}{B_0}= \frac{1}{\rho^3}\left(2 \cos \theta \hat \rho + \sin \theta \hat \theta \right)$$

where the dimensionless radial coordinate is $\rho := r/R_E$, and $\hat \rho = \hat r$ is the same radial unit vector. 

Using the standard spherical to cartesian transformation, we have ...

In [2]:
T = 1e3 #Kinetic energy in eV
RE = 6371 #Radius of earth in km
Lat = 66 #Latitude in degrees
ME = 8.05e22 #Magnetic moment of earth in Am^2
m = 1.672e-27 #Proton mass in kg

In [6]:
def B(x,y,z):
    
    Bfield = np.zeros(3,dtype=np.float64)
    
    Bfield[0] = 0